In [25]:
import csv
import numpy as np
import Loader
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.losses import MeanSquaredError

In [2]:
compoundsTrain, smilesTrain, labelsTrain, compoundDataTrain, activitiesTrain = Loader.getTrain(defaultValue=0)
compoundsTest, smilesTest, labelsTest, compoundDataTest, activitiesTest = Loader.getTest(defaultValue=0)
compoundsValidate, smilesValidate, labelsValidate, compoundDataValidate, activitiesValidate = Loader.getValidate(defaultValue=0)

#print(labelsTrain)
#print(compoundsTrain)
#print(smilesTrain)
#print(activitiesTrain)

#for i in range(len(labelsTrain)):
#    print(labelsTrain[i] + ": ", compoundDataTrain[0,i])

def toClassification(y): # The resulting array will contain values of -1 if it is below 4.5 and 1 if it is above
    y = np.array(y)
    classification = (y.astype(float)>4.5).astype(int)
    return classification * 2 - 1

print(len(compoundsTrain), len(compoundsTest), len(compoundsValidate))

2717 340 340


In [32]:
print(compoundDataTrain, toClassification(activitiesTrain)[toClassification(activitiesTrain) == 1])

[[-7.9 -7.9 -7.4 ...  0.   0.   0. ]
 [-5.4 -5.4 -5.4 ...  0.   0.   0. ]
 [-7.2 -7.2 -7.1 ...  0.   0.   0. ]
 ...
 [-6.3 -6.2 -6.1 ...  0.   0.   0. ]
 [-8.6 -8.2 -8.1 ...  0.   0.   0. ]
 [-6.5 -6.4 -6.4 ...  0.   0.   0. ]] [1 1 1 ... 1 1 1]


In [3]:
for x in compoundDataTrain:
    for val in x:
        if val == 10e10:
            print(val)

100000000000.0
100000000000.0


In [23]:
model = Sequential()
model.add(Dense(50, input_dim=np.shape(compoundDataTrain)[1], activation='relu', kernel_regularizer = keras.regularizers.L2(0.001)))
model.add(Dense(100, activation='relu', kernel_regularizer = keras.regularizers.L2(0.001)))
model.add(Dense(50, activation='relu', kernel_regularizer = keras.regularizers.L2(0.001)))
model.add(Dense(75, activation='relu', kernel_regularizer = keras.regularizers.L2(0.001)))
model.add(Dense(20, activation='relu', kernel_regularizer = keras.regularizers.L2(0.001)))
model.add(Dense(100, activation='relu', kernel_regularizer = keras.regularizers.L2(0.001)))
model.add(Dense(100, activation='relu', kernel_regularizer = keras.regularizers.L2(0.001)))
model.add(Dense(1, activation='softmax', kernel_regularizer = keras.regularizers.L2(0.001)))

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['MeanSquaredError'])

history = model.fit(compoundDataTrain, activitiesTrain,validation_data = (compoundDataTest, activitiesTest), epochs=100, batch_size=32)

Epoch 1/100
85/85 [==============================] - 2s 9ms/step - loss: 16.5646 - mean_squared_error: 16.3299 - val_loss: 16.4323 - val_mean_squared_error: 16.3337
Epoch 2/100
85/85 [==============================] - 1s 7ms/step - loss: 16.3791 - mean_squared_error: 16.3299 - val_loss: 16.3525 - val_mean_squared_error: 16.3337
Epoch 3/100
85/85 [==============================] - 0s 5ms/step - loss: 16.3388 - mean_squared_error: 16.3299 - val_loss: 16.3368 - val_mean_squared_error: 16.3337
Epoch 4/100
85/85 [==============================] - 0s 5ms/step - loss: 16.3313 - mean_squared_error: 16.3299 - val_loss: 16.3342 - val_mean_squared_error: 16.3337
Epoch 5/100
85/85 [==============================] - 0s 5ms/step - loss: 16.3301 - mean_squared_error: 16.3299 - val_loss: 16.3338 - val_mean_squared_error: 16.3337
Epoch 6/100
85/85 [==============================] - 0s 5ms/step - loss: 16.3299 - mean_squared_error: 16.3299 - val_loss: 16.3337 - val_mean_squared_error: 16.3337
Epoch 7/10

85/85 [==============================] - 0s 4ms/step - loss: 16.3299 - mean_squared_error: 16.3299 - val_loss: 16.3337 - val_mean_squared_error: 16.3337
Epoch 51/100
85/85 [==============================] - 0s 6ms/step - loss: 16.3299 - mean_squared_error: 16.3299 - val_loss: 16.3337 - val_mean_squared_error: 16.3337
Epoch 52/100
85/85 [==============================] - 0s 4ms/step - loss: 16.3299 - mean_squared_error: 16.3299 - val_loss: 16.3337 - val_mean_squared_error: 16.3337
Epoch 53/100
85/85 [==============================] - 1s 7ms/step - loss: 16.3299 - mean_squared_error: 16.3299 - val_loss: 16.3337 - val_mean_squared_error: 16.3337
Epoch 54/100
85/85 [==============================] - 0s 5ms/step - loss: 16.3299 - mean_squared_error: 16.3299 - val_loss: 16.3337 - val_mean_squared_error: 16.3337
Epoch 55/100
85/85 [==============================] - 1s 8ms/step - loss: 16.3299 - mean_squared_error: 16.3299 - val_loss: 16.3337 - val_mean_squared_error: 16.3337
Epoch 56/100
85/8

85/85 [==============================] - 0s 4ms/step - loss: 16.3299 - mean_squared_error: 16.3299 - val_loss: 16.3337 - val_mean_squared_error: 16.3337
Epoch 100/100
85/85 [==============================] - 1s 7ms/step - loss: 16.3299 - mean_squared_error: 16.3299 - val_loss: 16.3337 - val_mean_squared_error: 16.3337


In [28]:
model = Sequential()
model.add(Dense(50, input_dim=np.shape(compoundDataTrain)[1], activation='softmax', kernel_regularizer = keras.regularizers.L2(0.001)))
model.add(Dense(100, activation='softmax', kernel_regularizer = keras.regularizers.L2(0.001)))
model.add(Dense(50, activation='relu', kernel_regularizer = keras.regularizers.L2(0.001)))
model.add(Dense(75, activation='relu', kernel_regularizer = keras.regularizers.L2(0.001)))
model.add(Dense(100, activation='relu', kernel_regularizer = keras.regularizers.L2(0.001)))
model.add(Dense(75, activation='relu', kernel_regularizer = keras.regularizers.L2(0.001)))
model.add(Dense(100, activation='relu', kernel_regularizer = keras.regularizers.L2(0.001)))
model.add(Dense(1, activation='softmax', kernel_regularizer = keras.regularizers.L2(0.001)))

model.compile(loss='MeanSquaredError', optimizer='adam', metrics=['accuracy'])

history = model.fit(compoundDataTrain, toClassification(activitiesTrain),validation_data = (compoundDataTest, toClassification(activitiesTest)), epochs=100, batch_size=32)

Epoch 1/100
85/85 [==============================] - 2s 9ms/step - loss: 1.7991 - accuracy: 0.6198 - val_loss: 1.6777 - val_accuracy: 0.6088
Epoch 2/100
85/85 [==============================] - 1s 7ms/step - loss: 1.5753 - accuracy: 0.6198 - val_loss: 1.5839 - val_accuracy: 0.6088
Epoch 3/100
85/85 [==============================] - 1s 9ms/step - loss: 1.5293 - accuracy: 0.6198 - val_loss: 1.5673 - val_accuracy: 0.6088
Epoch 4/100
85/85 [==============================] - 1s 10ms/step - loss: 1.5219 - accuracy: 0.6198 - val_loss: 1.5650 - val_accuracy: 0.6088
Epoch 5/100
85/85 [==============================] - 1s 7ms/step - loss: 1.5209 - accuracy: 0.6198 - val_loss: 1.5647 - val_accuracy: 0.6088
Epoch 6/100
85/85 [==============================] - 1s 8ms/step - loss: 1.5208 - accuracy: 0.6198 - val_loss: 1.5647 - val_accuracy: 0.6088
Epoch 7/100
85/85 [==============================] - 1s 8ms/step - loss: 1.5208 - accuracy: 0.6198 - val_loss: 1.5647 - val_accuracy: 0.6088
Epoch 8/100


Epoch 59/100
85/85 [==============================] - 0s 4ms/step - loss: 1.5208 - accuracy: 0.6198 - val_loss: 1.5647 - val_accuracy: 0.6088
Epoch 60/100
85/85 [==============================] - 0s 5ms/step - loss: 1.5208 - accuracy: 0.6198 - val_loss: 1.5647 - val_accuracy: 0.6088
Epoch 61/100
85/85 [==============================] - 1s 7ms/step - loss: 1.5208 - accuracy: 0.6198 - val_loss: 1.5647 - val_accuracy: 0.6088
Epoch 62/100
85/85 [==============================] - 0s 5ms/step - loss: 1.5208 - accuracy: 0.6198 - val_loss: 1.5647 - val_accuracy: 0.6088
Epoch 63/100
85/85 [==============================] - 1s 8ms/step - loss: 1.5208 - accuracy: 0.6198 - val_loss: 1.5647 - val_accuracy: 0.6088
Epoch 64/100
85/85 [==============================] - 0s 5ms/step - loss: 1.5208 - accuracy: 0.6198 - val_loss: 1.5647 - val_accuracy: 0.6088
Epoch 65/100
85/85 [==============================] - 0s 4ms/step - loss: 1.5208 - accuracy: 0.6198 - val_loss: 1.5647 - val_accuracy: 0.6088
Epoch 

In [ ]:
csvfile=open("../Data/trainDataDescriptors.csv", newline='',  encoding='UTF-8')
rd = csv.reader(csvfile, delimiter=',')
trainDescriptors=[]
for lv in rd:
    trainDescriptors.insert(1,lv)
headersDescriptors = trainDescriptors[0]
trainDescriptors = trainDescriptors[1:]

csvfile=open("../Data/trainDockingData.csv", newline='',  encoding='UTF-8')
rd = csv.reader(csvfile, delimiter=',')
trainDocking=[]
for lv in rd:
    trainDocking.insert(1,lv)
headersDocking = trainDocking[0]
trainDocking = trainDocking[1:]


csvfile=open("../Data/testDataDescriptors.csv", newline='',  encoding='UTF-8')
rd = csv.reader(csvfile, delimiter=',')
testDescriptors=[]
for lv in rd:
    testDescriptors.insert(1,lv)
testDescriptors = testDescriptors[1:]

csvfile=open("../Data/testDockingData.csv", newline='',  encoding='UTF-8')
rd = csv.reader(csvfile, delimiter=',')
testDocking=[]
for lv in rd:
    testDocking.insert(1,lv)
testDocking = testDocking[1:]


testing = False
if not testing:
    # Removing the compound id from the middle of the docking data
    trainDocking = np.delete(trainDocking, headersDocking.index("compound_id"), axis=1)
    testDocking = np.delete(testDocking, headersDocking.index("compound_id"), axis=1)

    headersDocking.remove("compound_id")

print(np.shape(trainDocking),np.shape(trainDescriptors),np.shape(testDocking), np.shape(testDescriptors))

trainActivity = []
testActivity = []



# Verifying They Are in the Same Order

In [ ]:
#Only works if compound_id is not deleted above
if testing:
    sameTrain = True
    for i in range(len(trainDocking)):
        sameTrain = sameTrain and (trainDocking[i][headersDocking.index("compound_id")] == trainDescriptors[i][headersDescriptors.index("compound_id")])
        print(trainDocking[i][headersDocking.index("compound_id")],trainDescriptors[i][headersDescriptors.index("compound_id")])
    print(sameTrain)

    sameTest = True
    for i in range(len(testDocking)):
        sameTest = sameTest and (testDocking[i][headersDocking.index("compound_id")] == testDescriptors[i][headersDescriptors.index("compound_id")])
        #print(trainDocking[i][headersDocking.index("compound_id")] == trainDescriptors[i][headersDescriptors.index("compound_id")])
    print(sameTest)

# Combining the Two Datasets

In [ ]:
# Because they are one to one, we can easily combine the datasets together immediately


#print(len(trainDocking[0]))
#Taking out the activity score, as it will become the desired output of the model
if len(trainDocking[0]) == 22:
    trainActivity = np.array(trainDocking)[:,headersDocking.index("activity")]
    testActivity = np.array(testDocking)[:,headersDocking.index("activity")]

    trainDocking = np.delete(trainDocking, headersDocking.index("activity"), axis=1)
    testDocking = np.delete(testDocking, headersDocking.index("activity"), axis=1)
    
    headersDocking.remove("activity")

trainX = np.append(trainDescriptors, trainDocking,1)
trainY = trainActivity

testX = np.append(testDescriptors, testDocking,1)
testY = testActivity
#Appending for some reason reverses both arrays, so the index is actually the row 2717 - ind in excel (idk why)
print(trainY[300], trainX[300])

In [ ]:
trainY = trainActivity
trainY = toClassification(trainY)
print(trainActivity[trainY == 1]) #Gives values of > 4.5